# Cinetel Data Cleaning

Cinetel data scraped daily from their boxoffice published everyday at this [link](https://www.cinetel.it/pages/boxoffice.php?edperiodo=aWVyaQ==).

## Reading In Files

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"E:\data_analysis_python\cinetel\cinetel_.csv")
df1.head()

,Pos.,Titolo,Prima Progr.,Nazione,Distribuzione,Incasso,Presenze,Incasso al 16/11/2023,Presenze al 16/11/2023,2023-11-17 15:31:50.351708
0,1,C'E' ANCORA DOMANI,'2023-10-26'26/10/2023,ITA,VISION DISTRIBUTION,449991.06,66700,15231636.85,2202169,2023-11-17 15:31:50.351708
1,2,HUNGER GAMES - LA BALLATA DELL'USIGNOLO E DEL ...,'2023-11-15'15/11/2023,USA,MEDUSA FILM S.P.A.,238466.31,32469,584902.45,79311,2023-11-17 15:31:50.351708
2,3,THE MARVELS,'2023-11-08'08/11/2023,USA,WALT DISNEY S.M.P. ITALIA,60914.35,8628,2260142.44,305291,2023-11-17 15:31:50.351708
3,4,THANKSGIVING,'2023-11-16'16/11/2023,USA,EAGLE PICTURES S.P.A.,41232.57,5709,41232.57,5709,2023-11-17 15:31:50.351708
4,5,THE OLD OAK,'2023-11-16'16/11/2023,FRA,LUCKY RED DISTRIB.,36605.89,5879,56540.79,8609,2023-11-17 15:31:50.351708


---

In [3]:
df1 = pd.DataFrame(df1)

## Cleaning the dataframe

To clean the dataframe I decided to drop duplicate values and columns that aren't relevant to my analysis, such as `df1['Incasso al 16/11/2023']` and `df1['Presenze al 16/11/2023]`. I then decided to rename the existing columns using the snake_case rule in order to make the analysis easier.

In [4]:
df1 = df1.drop_duplicates()

In [5]:
# df1 = df1.drop(columns= ['Incasso al 16/11/2023', 'Presenze al 16/11/2023'])

# at first i thought these columns weren't relevant
# making visualizations, i realized i need the data to have accurate insights on movies that started their screenings before i started to scrape the data

In [6]:
df1 = df1.rename(columns = {'Pos.':'daily_rank', 'Titolo' : 'title', 'Prima Progr.' : 'first_screening_date', 'Nazione' : 'nation', 'Distribuzione' : 'distribution', 'Incasso' : 'daily_takings', 'Presenze' : 'daily_attendance','Incasso al 16/11/2023' : 'total_takings',  'Presenze al 16/11/2023' : 'total_attendance', '2023-11-17 15:31:50.351708' : 'date_pulled'})

In [7]:
# dropping the extra 'header row' that is imported everyday when the .csv file gets updated
df1 = df1[df1['daily_rank'].str.contains('Pos.')==False]

In [8]:
# making the 'title' and 'distribution' entries consistent by appling the title format
df1['title'] = df1['title'].apply(lambda x: x.title())
df1['distribution'] = df1['distribution'].apply(lambda x: x.title())

In [9]:
# cleaning the 'first_screening_date' entries
df1['first_screening_date'] = df1['first_screening_date'].str[12:]

In [10]:
# cleaning the column showing the date the data was pulled from the site
df1['date_pulled'] = pd.to_datetime(df1['date_pulled'])
df1['date_pulled'] = pd.to_datetime(df1['date_pulled']).dt.date

#extra tries:
#df1['date_pulled'] = df1['date_pulled'].str[:10]
#df1['date_pulled'] = df1['date_pulled'].str.replace('[^a-zA-Z0-9]', '', regex = True)
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: str(x))
#df1['date_pulled'] = df1['date_pulled'].apply(lambda x: x[0:2] + '/' + x[2:4] + '/' + x[4:8])

In [11]:
from datetime import timedelta, datetime

In [12]:
df1['first_screening_date']

0       26/10/2023
1       15/11/2023
2       08/11/2023
3       16/11/2023
4       16/11/2023
           ...    
1281    25/01/2024
1282    29/02/2024
1283    23/08/2023
1284    14/03/2024
1285    22/02/2024
Name: first_screening_date, Length: 1170, dtype: object

In [13]:
# formatting
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')
df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date']).dt.date

C:\Users\Franci\AppData\Local\Temp\ipykernel_26756\2741308956.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df1['first_screening_date'] = pd.to_datetime(df1['first_screening_date'], errors='coerce')


In [14]:
df1['daily_takings'] = df1['daily_takings'].astype(float)
df1['daily_attendance'] = df1['daily_attendance'].astype(int)
df1['daily_rank'] = df1['daily_rank'].astype(int)
df1['total_takings'] = df1['total_takings'].astype(float)
df1['total_attendance'] = df1['total_attendance'].astype(int)

---

## Data manipulation

I decided to create new columns to get information I may need later on during the analysis.

In [15]:
# calculating the number of days of screenings for each movie 
df1['screening_days'] = datetime.today().date() - df1['first_screening_date']
df1['screening_days'] = df1['screening_days'].map(lambda x: str(x)[:-14])

In [16]:
df1['screening_days']

0       143
1       123
2       130
3       122
4       122
       ... 
1281     52
1282     17
1283    207
1284      3
1285     24
Name: screening_days, Length: 1170, dtype: object

In [17]:
#df1['screening_days'] = df1['screening_days'].str.strip()
df1['screening_days'] = df1['screening_days'].replace('', '0')
df1['screening_days'] = df1['screening_days'].astype(int) + 1

In [18]:
# creating a column to show the date the 'daily_takings' and 'daily_attendace' refer to
df1['date'] = df1['date_pulled'] - timedelta(days=1)
df1['date'] = pd.to_datetime(df1['date'])

In [19]:
# creating a column showing the day of the week. it'll be useful for further analysis
df1['day_of_week'] = df1['date'].dt.day_name()

In [20]:
df1['avg_ticket_price'] = df1['daily_takings'] / df1['daily_attendance']

In [21]:
pd.set_option('display.max_rows', 8)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df1

,daily_rank,title,first_screening_date,nation,distribution,daily_takings,daily_attendance,total_takings,total_attendance,date_pulled,screening_days,date,day_of_week,avg_ticket_price
0,1,C'E' Ancora Domani,2023-10-26,ITA,Vision Distribution,449991.06,66700,15231636.85,2202169,2023-11-17,144,2023-11-16,Thursday,6.75
1,2,Hunger Games - La Ballata Dell'Usignolo E Del ...,2023-11-15,USA,Medusa Film S.P.A.,238466.31,32469,584902.45,79311,2023-11-17,124,2023-11-16,Thursday,7.34
2,3,The Marvels,2023-11-08,USA,Walt Disney S.M.P. Italia,60914.35,8628,2260142.44,305291,2023-11-17,131,2023-11-16,Thursday,7.06
3,4,Thanksgiving,2023-11-16,USA,Eagle Pictures S.P.A.,41232.57,5709,41232.57,5709,2023-11-17,123,2023-11-16,Thursday,7.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,7,La Sala Professori (The Teachers' Lounge),2024-02-29,DEU,Lucky Red Distrib.,76286.46,11281,818409.56,126416,2024-03-17,18,2024-03-16,Saturday,6.76
1283,8,Oppenheimer,2023-08-23,USA,Universal S.R.L.,75867.17,10925,28325024.97,3798530,2024-03-17,208,2024-03-16,Saturday,6.94
1284,9,Force Of Nature - Oltre L'Inganno,2024-03-14,USA,Notorious Pict. S.P.A.,53277.40,7212,87019.73,12168,2024-03-17,4,2024-03-16,Saturday,7.39
1285,10,Emma E Il Giaguaro Nero (Le Dernier Jaguar),2024-02-22,FRA,01 Distribution,43852.62,6598,1567292.64,233816,2024-03-17,25,2024-03-16,Saturday,6.65


---

## Export as .csv

In [22]:
import os

In [23]:
clean_file_name = 'cinetel_clean.csv'
clean_file_path = os.path.join(os.getcwd(), clean_file_name)

In [24]:
df1.to_csv(clean_file_path, index=False)
print(f"Il DataFrame è stato esportato con successo in {clean_file_path}.")

Il DataFrame è stato esportato con successo in e:\data_analysis_python\cinetel\cinetel_clean.csv.


---

## Data Exploration

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
pd.set_option('display.max_rows', 8)

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1170 entries, 0 to 1285
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daily_rank            1170 non-null   int32         
 1   title                 1170 non-null   object        
 2   first_screening_date  1163 non-null   object        
 3   nation                1170 non-null   object        
 4   distribution          1170 non-null   object        
 5   daily_takings         1170 non-null   float64       
 6   daily_attendance      1170 non-null   int32         
 7   total_takings         1170 non-null   float64       
 8   total_attendance      1170 non-null   int32         
 9   date_pulled           1170 non-null   object        
 10  screening_days        1170 non-null   int32         
 11  date                  1170 non-null   datetime64[ns]
 12  day_of_week           1170 non-null   object        
 13  avg_ticket_price      1

In [28]:
df1.describe()

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,date,avg_ticket_price
count,1170.00,1170.00,1170.00,1170.00,1170.00,1170.00,1170,1170.00
mean,5.50,144779.15,20118.27,4683679.42,665473.21,84.78,2024-01-12 19:53:50.769230592,6.93
min,1.00,6272.70,864.00,11161.11,1470.00,1.00,2023-11-16 00:00:00,3.70
25%,3.00,30713.81,4672.25,487425.49,68362.50,53.00,2023-12-15 00:00:00,6.36
50%,5.50,71368.05,10468.00,1723117.19,249265.00,88.00,2024-01-11 00:00:00,6.94
75%,8.00,173831.83,24085.50,4546513.96,647550.00,115.00,2024-02-10 00:00:00,7.39
max,10.00,1544231.00,211764.00,36581221.80,5399550.00,914.00,2024-03-16 00:00:00,21.18
std,2.87,192650.83,25783.73,8188095.59,1193236.16,62.81,NaN,1.08


In [29]:
df1.isnull().sum()

daily_rank              0
title                   0
first_screening_date    7
nation                  0
                       ..
screening_days          0
date                    0
day_of_week             0
avg_ticket_price        0
Length: 14, dtype: int64

In [30]:
df1.nunique()

daily_rank                10
title                     98
first_screening_date      52
nation                    12
                        ... 
screening_days            53
date                     113
day_of_week                7
avg_ticket_price        1130
Length: 14, dtype: int64

In [31]:
df1.corr(numeric_only=True)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
daily_rank,1.00,-0.55,-0.56,-0.07,-0.06,0.12,-0.12
daily_takings,-0.55,1.00,1.00,0.12,0.12,0.03,0.24
daily_attendance,-0.56,1.00,1.00,0.15,0.14,0.03,0.19
total_takings,-0.07,0.12,0.15,1.00,1.00,0.32,-0.18
total_attendance,-0.06,0.12,0.14,1.00,1.00,0.32,-0.19
screening_days,0.12,0.03,0.03,0.32,0.32,1.00,-0.04
avg_ticket_price,-0.12,0.24,0.19,-0.18,-0.19,-0.04,1.00


In [32]:
df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,4.66,247731.30,36798.43,30931682.52,4533157.52,144.00,6.24
Oppenheimer,8.50,33550.84,5001.25,28244585.53,3786716.50,208.00,6.49
Wonka,3.85,368953.34,48842.15,9842301.36,1311072.30,95.00,7.32
Dune,9.00,25782.87,3338.75,7432259.33,1035522.25,914.00,7.53
...,...,...,...,...,...,...,...
Genoa. Comunque E Ovunque,9.00,24779.00,2726.00,34295.70,3834.00,49.00,9.09
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,180.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,93.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,112.00,9.62


In [33]:
df2 = df1.groupby('title').mean(numeric_only=True).sort_values(by = 'total_attendance', ascending= False)
df2

,daily_rank,daily_takings,daily_attendance,total_takings,total_attendance,screening_days,avg_ticket_price
title,,,,,,,
C'E' Ancora Domani,4.66,247731.30,36798.43,30931682.52,4533157.52,144.00,6.24
Oppenheimer,8.50,33550.84,5001.25,28244585.53,3786716.50,208.00,6.49
Wonka,3.85,368953.34,48842.15,9842301.36,1311072.30,95.00,7.32
Dune,9.00,25782.87,3338.75,7432259.33,1035522.25,914.00,7.53
...,...,...,...,...,...,...,...
Genoa. Comunque E Ovunque,9.00,24779.00,2726.00,34295.70,3834.00,49.00,9.09
Royal Opera House 2023/2024,7.00,19784.50,1916.00,35873.50,3496.00,180.00,10.33
Seventeen Tour Follow To Japan: Live Viewing,9.00,66089.87,3121.00,66089.87,3121.00,93.00,21.18
Casanova Operapop - Il Film,9.00,12093.47,1256.33,25578.93,2715.33,112.00,9.62
